In [1]:
"""
Feature: resp_rate
Extract max respiratory rate from chartevents (ICU) and vitalsign (ED)
"""
import pandas as pd
import os
from config_paths import *
from utils import drop_columns

In [2]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load ICU chartevents in chunks
resp_itemids = [220210, 224689, 224690]
resp_max_list = []

try:
    chunk_iter = pd.read_csv(
        os.path.join(icu_path, "chartevents.csv"),
        usecols=["subject_id", "hadm_id", "itemid", "valuenum"],
        chunksize=1000000,
        low_memory=False
    )
    
    for chunk in chunk_iter:
        rr = chunk[chunk["itemid"].isin(resp_itemids)].copy()
        rr = rr.dropna(subset=["valuenum"])
        rr = rr[(rr["valuenum"] >= 3) & (rr["valuenum"] <= 80)]
        
        if not rr.empty:
            rr_max_chunk = rr.groupby("hadm_id")["valuenum"].max()
            resp_max_list.append(rr_max_chunk)
    
    if resp_max_list:
        resp_max_icu = (
            pd.concat(resp_max_list)
            .groupby("hadm_id")
            .max()
            .reset_index()
            .rename(columns={"valuenum": "resp_rate_icu"})
        )
    else:
        resp_max_icu = pd.DataFrame(columns=["hadm_id", "resp_rate_icu"])
except:
    resp_max_icu = pd.DataFrame(columns=["hadm_id", "resp_rate_icu"])

# Load ED vitalsign
try:
    vitalsign = pd.read_csv(os.path.join(ed_path, "vitalsign.csv"))
    ed_stays = pd.read_csv(os.path.join(ed_path, "edstays.csv"))
    
    ed_vitals_rr = ed_stays.merge(
        vitalsign[["subject_id", "stay_id", "resprate"]],
        on=["subject_id", "stay_id"],
        how="left"
    )
    
    resp_max_ed = (
        ed_vitals_rr.groupby(["subject_id", "hadm_id"])["resprate"]
        .max()
        .reset_index()
        .rename(columns={"resprate": "resp_rate_ed"})
    )
except:
    resp_max_ed = pd.DataFrame(columns=["subject_id", "hadm_id", "resp_rate_ed"])

# Merge
df = df.merge(resp_max_icu, on="hadm_id", how="left")
if not resp_max_ed.empty:
    df = df.merge(resp_max_ed, on=["subject_id", "hadm_id"], how="left")

# Combine: prefer ICU, then ED
df["resp_rate"] = df["resp_rate_icu"].fillna(df.get("resp_rate_ed", pd.Series()))

# Drop intermediate columns
df.drop(columns=["resp_rate_icu", "resp_rate_ed"], errors="ignore", inplace=True)

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nTotal rows: {len(df)}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")
print(f"Feature 'resp_rate' added. Non-null count: {df['resp_rate'].notna().sum()}")
print(f"Dataset shape: {df.shape}")



Total rows: 158020
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 95)
Feature 'resp_rate' added. Non-null count: 110953
Dataset shape: (158020, 95)


In [3]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [4]:
# cols_to_drop = [
#  "resp_rate"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)